In [1]:
import pickle
import pandas as pd

In [36]:
#Convert match dictionary data to match frame dataframe with features
def extract_match_df(matches):
    win_col = []
    match_id_col = []
    blue_turr_down_col = []
    red_turr_down_col = []
    blue_team_gold_col = []
    red_team_gold_col = []
    blue_team_level_col = []
    red_team_level_col = []
    blue_team_damage_col = []
    red_team_damage_col = []
    blue_team_kills_col = []
    red_team_kills_col = []
    blue_team_drags_col = []
    red_team_drags_col = []
    blue_team_barons_col = []
    red_team_barons_col = []
    blue_team_inhibs_col = []
    red_team_inhibs_col = []
    frame_col = []
    blue_team_participants = ['1','2','3','4','5']
    red_team_participants = ['6', '7', '8', '9', '10']
    
    for match in matches:
        match_id = match[0]['metadata']['matchId']
        try:
            win = match[0]['info']['teams'][0]['win']
        except:
            pass
        blue_turrets_down = 0
        red_turrets_down = 0
        blue_team_kills = 0
        red_team_kills = 0
        blue_team_drags = 0
        red_team_drags = 0
        blue_team_barons = 0
        red_team_barons = 0
        blue_team_inhibs = 0
        red_team_inhibs = 0
        frame_num = 1
        for frame in match[1]['info']['frames'][1:]:
            for event in frame['events']:
                if event['type'] == 'BUILDING_KILL':
                    if event['buildingType'] == 'TOWER_BUILDING':
                        if event['teamId'] == 100:
                            blue_turrets_down += 1
                        if event['teamId'] == 200:
                            red_turrets_down += 1
                    if event['buildingType'] == 'INHIBITOR_BUILDING':
                        if str(event['killerId']) in blue_team_participants:
                            blue_team_inhibs += 1
                        if str(event['killerId']) in red_team_participants:
                            red_team_inhibs += 1
                elif event['type'] == 'CHAMPION_KILL':
                    if str(event['killerId']) in blue_team_participants:
                        blue_team_kills += 1
                    if str(event['killerId']) in red_team_participants:
                        red_team_kills += 1
                elif event['type'] == 'ELITE_MONSTER_KILL':
                    if event['monsterType'] == 'DRAGON':
                        if str(event['killerId']) in blue_team_participants:
                            blue_team_drags += 1
                        if str(event['killerId']) in red_team_participants:
                            red_team_drags += 1
                    if event['monsterType'] == 'BARON_NASHOR':
                        if str(event['killerId']) in blue_team_participants:
                            blue_team_barons += 1
                        if str(event['killerId']) in red_team_participants:
                            red_team_barons += 1
                    
        
            blue_team_gold = 0
            red_team_gold = 0
            blue_team_level = 0
            red_team_level = 0
            blue_team_damage = 0
            red_team_damage = 0
        
            #Blue team aggregate stats
            for participant_id in blue_team_participants:
                participant = frame['participantFrames'][participant_id]
                blue_team_gold += participant['totalGold']
                blue_team_level += participant['level']
                blue_team_damage += participant['damageStats']['totalDamageDoneToChampions']
            #Red team aggregate stats    
            for participant_id in red_team_participants:
                participant = frame['participantFrames'][participant_id]
                red_team_gold += participant['totalGold']
                red_team_level += participant['level']
                red_team_damage += participant['damageStats']['totalDamageDoneToChampions']                                  
                                          
            #Add Observations to lists                
            win_col.append(win)
            match_id_col.append(match_id)
            blue_turr_down_col.append(blue_turrets_down)
            red_turr_down_col.append(red_turrets_down)
            blue_team_gold_col.append(blue_team_gold)
            red_team_gold_col.append(red_team_gold)
            blue_team_level_col.append(blue_team_level)
            red_team_level_col.append(red_team_level)
            blue_team_damage_col.append(blue_team_damage)
            red_team_damage_col.append(red_team_damage)
            blue_team_kills_col.append(blue_team_kills)
            red_team_kills_col.append(red_team_kills)
            blue_team_drags_col.append(blue_team_drags)
            red_team_drags_col.append(red_team_drags)
            blue_team_barons_col.append(blue_team_barons)
            red_team_barons_col.append(red_team_barons)
            blue_team_inhibs_col.append(blue_team_inhibs)
            red_team_inhibs_col.append(red_team_inhibs)
            frame_col.append(frame_num)
    
            frame_num += 1
    matches_df = pd.DataFrame(list(zip(match_id_col, win_col, frame_col,blue_turr_down_col, red_turr_down_col, blue_team_gold_col, red_team_gold_col, 
                                       blue_team_level_col, red_team_level_col, blue_team_damage_col, red_team_damage_col,
                                       blue_team_kills_col, red_team_kills_col, blue_team_drags_col, red_team_drags_col,
                                       blue_team_barons_col, red_team_barons_col, blue_team_inhibs_col, red_team_inhibs_col
                                      )),
               columns =['MatchId', 'Blue_Win', 'Frame_Minute', 'Blue_Turret_Down', 'Red_Turret_Down', 'Blue_Gold', 'Red_Gold', 'Blue_Level', 'Red_Level', 
                         'Blue_Damage', 'Red_Damage', 'Blue_Kills', 'Red_Kills', 'Blue_Drags', 'Red_Drags', 'Blue_Barons', 'Red_Barons',
                         'Blue_Inhibs', 'Red_Inhibs'
                        ])
    return matches_df

In [37]:
with open('./data/matches1.pickle', 'rb') as handle:
    matches1 = pickle.load(handle)
matches_df = extract_match_df(matches1)

In [41]:
for i in range(2,11):
    with open('./data/matches'+str(i)+'.pickle', 'rb') as handle:
        matches_temp = pickle.load(handle)
    matches_temp_df = extract_match_df(matches_temp)
    matches_df = pd.concat([matches_df, matches_temp_df], axis=0)

In [42]:
len(matches_df)

323248

In [43]:
matches_df.to_csv('matches.csv',index=False)